In [1]:
import logging
import logging.handlers
import os
import io
import pandas as pd
import numpy as np
import requests
import json
from chessdotcom import get_player_profile, get_player_stats, get_player_game_archives
import chess.pgn
from converter.pgn_data import PGNData
import time
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

In [3]:
"""
'./students.json'  should be replaced with database in the future.

That is the goal of this sprint
"""

"\n'./students.json'  should be replaced with database in the future.\n\nThat is the goal of this sprint\n"

In [4]:
def student_df(student_data):
    """
    input:
    student_data - dictionary, where key is the class and value is a list of username of each student
    
    output:
    a dataframe having two columns - class, student class; username, student username
    """
    df = pd.DataFrame({'Keys': list(student_data.keys()), 'Values': list(student_data.values())})
    df = df.explode(column='Values').reset_index(drop=True)
    df.rename(columns = {'Keys':'class', 'Values':'username'}, inplace = True)
    return df

In [5]:
user = {
            "Tianmin Lyu":"tianminlyu"
            }

In [6]:
student_df(user)

,class,username
0,Tianmin Lyu,tianminlyu


In [7]:
def last_n_month(n):
    """
    return the month as yyyy/mm format of the past n months from now
    input -
    n: number of months from past
    
    output -
    a list of month with yyyy/mm format
    """
    months_lst = []
    for num in range(n):
        months = date.today() + relativedelta(months=-num)
        if months.month <= 9:
            months_lst.append(str(months.year) + "/"+ "0" + str(months.month))
        else:
            months_lst.append(str(months.year) + "/"+ str(months.month))
    return months_lst

In [8]:
last_n_month(2)

['2023/09', '2023/08']

In [9]:
def get_user_archives(username, months):
    """
    get archive monthly files of specific chess.com player
    input:
    username - username of the chess.com player
    months - target months that we want to get the archives
    
    output:
    target_month - files of archives according to months parameter
    """
    archives = get_player_game_archives(username).json['archives']
    target_month = []
    for archive in archives:
        if archive[-7:] in months:
            target_month.append(archive)
    return target_month


In [28]:
url = "https://api.chess.com/pub/player/tianminlyu/games/archives"



user_agent = {'User-Agent': 'username: tianminlyu, email: tianminlyu@gmail.com'}

r = requests.get(url, headers = user_agent)


In [29]:
archive_data = r.json()


In [18]:
archive_date['archives']

['https://api.chess.com/pub/player/tianminlyu/games/2019/08',
 'https://api.chess.com/pub/player/tianminlyu/games/2020/05',
 'https://api.chess.com/pub/player/tianminlyu/games/2020/06',
 'https://api.chess.com/pub/player/tianminlyu/games/2022/07',
 'https://api.chess.com/pub/player/tianminlyu/games/2022/08',
 'https://api.chess.com/pub/player/tianminlyu/games/2022/09',
 'https://api.chess.com/pub/player/tianminlyu/games/2022/10',
 'https://api.chess.com/pub/player/tianminlyu/games/2022/11',
 'https://api.chess.com/pub/player/tianminlyu/games/2022/12',
 'https://api.chess.com/pub/player/tianminlyu/games/2023/01',
 'https://api.chess.com/pub/player/tianminlyu/games/2023/02',
 'https://api.chess.com/pub/player/tianminlyu/games/2023/03',
 'https://api.chess.com/pub/player/tianminlyu/games/2023/04',
 'https://api.chess.com/pub/player/tianminlyu/games/2023/05',
 'https://api.chess.com/pub/player/tianminlyu/games/2023/06',
 'https://api.chess.com/pub/player/tianminlyu/games/2023/07',
 'https:

In [16]:
def last_n_month(n):
    """
    purpose:
    return the month as yyyy/mm format of the past n months from now
    
    input -
    n: number of months from past
    
    output -
    a list of month with yyyy/mm format
    """
    months_lst = []
    for num in range(n):
        months = date.today() + relativedelta(months=-num)
        if months.month <= 9:
            months_lst.append(str(months.year) + "/"+ "0" + str(months.month))
        else:
            months_lst.append(str(months.year) + "/"+ str(months.month))
    return months_lst

In [17]:
last_n_month(2)

['2023/09', '2023/08']

In [36]:
def get_user_archives(username, nr_months):
    """
    purpose:
    get archive monthly files of specific chess.com player
    
    input:
    username - username of the chess.com player
    nr_months - integer, nummber of past months that we want to get the archives
    
    output:
    target_month - files of archives according to months parameter
    """
    url = "https://api.chess.com/pub/player/{username}/games/archives".format(username = username)
    user_agent = {'User-Agent': 'username: tianminlyu, email: tianminlyu@gmail.com'}
    archive_request = requests.get(url, headers = user_agent)
    archives = archive_request.json()['archives']
    past_months = last_n_month(nr_months)
    target_month = []
    for archive in archives:
        if archive[-7:] in past_months:
            target_month.append(archive)
    return target_month

In [37]:
get_user_archives('tianminlyu',2)

['https://api.chess.com/pub/player/tianminlyu/games/2023/08',
 'https://api.chess.com/pub/player/tianminlyu/games/2023/09']

In [38]:
archive_url = "https://api.chess.com/pub/player/tianminlyu/games/2023/09"

user_agent = {'User-Agent': 'username: tianminlyu, email: tianminlyu@gmail.com'}
archive_request = requests.get(archive_url, headers = user_agent)

archive_request

<Response [200]>

In [63]:
def get_archive_games(filename):
    """
    purpose:
    
    return games in one archive file
    
    input:
    filename - filename that contains game urls
    
    output: 
    """
    games = requests.get(filename,headers = user_agent).json()['games']
    return games

In [54]:
for game in archive_request.json()['games']:
    print(datetime.utcfromtimestamp(game['end_time']).strftime('%Y-%m-%d %H:%M:%S'))
    break

2023-09-01 09:39:15


In [56]:
students = ['yaohengli',
           'chessloverma',
           'chengliam',
           'emmaxli',
           'akfunchess66']

In [73]:
for student in students:
    print(student)
    archives = get_user_archives(student,2)
    for archive in archives[::-1]:
        games = get_archive_games(archive)
        for game in games[::-1]:
            print(datetime.utcfromtimestamp(game['end_time']).strftime('%Y-%m-%d %H:%M:%S'))
    print("------")

yaohengli
2023-09-08 02:05:22
2023-09-08 01:55:28
2023-09-07 01:49:40
2023-09-07 01:18:46
2023-09-07 01:07:47
2023-09-07 00:58:29
2023-09-06 03:28:22
2023-09-06 03:18:37
2023-09-06 03:04:13
2023-09-06 02:49:09
2023-09-04 23:33:47
2023-09-04 21:59:48
2023-09-04 21:57:52
2023-09-04 21:52:04
2023-09-04 05:05:49
2023-09-04 04:45:48
2023-09-04 04:30:31
2023-09-04 00:53:22
2023-09-04 00:42:41
2023-09-04 00:14:57
2023-09-03 23:36:03
2023-09-03 04:42:01
2023-09-03 04:32:53
2023-09-03 04:07:38
2023-09-03 02:15:17
2023-09-02 17:37:21
2023-09-02 00:47:09
2023-09-02 00:28:31
2023-09-01 04:23:44
2023-09-01 02:27:11
2023-09-01 02:05:22
2023-09-01 02:03:01
2023-09-01 01:45:35
2023-08-31 04:13:27
2023-08-31 03:52:01
2023-08-31 03:47:40
2023-08-31 02:13:09
2023-08-31 00:36:43
2023-08-31 00:28:09
2023-08-30 02:45:57
2023-08-29 00:53:30
2023-08-28 23:46:39
2023-08-26 03:16:52
2023-08-26 02:58:42
2023-08-25 23:40:13
2023-08-25 23:34:19
2023-08-25 23:08:03
2023-08-25 01:42:48
2023-08-25 01:05:52
2023-08-24

In [70]:
test = [1,2,3]

In [71]:
test[::-1]

[3, 2, 1]

In [ ]:
'https://api.chess.com/pub/player/yaohengli/games/2023/08'